In [1]:
import re, pandas as pd, numpy as np

In [2]:
collection = pd.read_csv('../wsb-catalog.csv')

In [3]:
def format_bibliographicCitation(cite):
    names = { 'schottlaender/4.0/': 'Schottlaender ', 'maynard-and-miles/': 'Maynard & Miles ', 'shoaf/': 'Shoaf ' }
    formatted_cite = []
    for prefix in names:
        p = re.compile(prefix + '([A-Z]\d+\w*)')
        numbers = p.findall(cite)
        if numbers:
            if len(numbers) > 2:
                formatted_cite.append(names[prefix] + ', '.join(numbers[:-1]) + ', and {}'.format(numbers[-1]))
            elif len(numbers) == 2:
                formatted_cite.append(names[prefix] + ' and '.join(numbers))
            else:
                formatted_cite.append(names[prefix] + numbers[0])
    if not formatted_cite:
        return ''
    else:
        return ', '.join(formatted_cite) + '.'

In [4]:
collection = collection.fillna('')

In [5]:
collection['image'] = collection['identifier'].astype('str').apply(lambda x: 'thumbnails/{}.jpg'.format(x)) 

In [6]:
collection['bibliographicCitation'] = collection['bibliographicCitation'].astype('str').apply(lambda x: format_bibliographicCitation(x)) 

In [7]:
collection['entry'] = np.where(collection['publisher'] == '', 'n.p.', collection['publisher'])
collection['entry'] = collection.apply(lambda x: '{}, {}. {} {}'.format(x['entry'], x['date'], x['description'], x['bibliographicCitation']), axis=1)

In [8]:
collection['mdTableRow'] = collection.apply(
    lambda x: '|![{}]({})|{}|{}|{}|'.format(x['identifier'], x['image'], x['creator'], x['title'], x['entry']),
    axis=1)

In [9]:
table_header = ['|image|creator|title|description|',
                '|---|---|---|---|']

In [10]:
table_body = pd.Series(collection['mdTableRow']).tolist()

In [11]:
table = table_header + table_body

In [12]:
with open('../docs/index.md', 'w') as outfile:
    for line in table:
        outfile.write(line + '\n')